# **Step 1:Mounting Drive**

In [1]:

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


# **Step 2: Load and Preprocess the Data**
First, we'll load the MovieLens dataset and preprocess it.

In [2]:
import pandas as pd

# Load the dataset
ratings = pd.read_csv('/content/drive/MyDrive/ml-100k/ml-100k/u.data', sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
movies = pd.read_csv('/content/drive/MyDrive/ml-100k/ml-100k/u.item', sep='|', encoding='latin-1', usecols=[0, 1], names=['item_id', 'title'])

# Merge the datasets
data = pd.merge(ratings, movies, on='item_id')

# Display the first few rows
print(data.head())


   user_id  item_id  rating  timestamp         title
0      196      242       3  881250949  Kolya (1996)
1       63      242       3  875747190  Kolya (1996)
2      226      242       5  883888671  Kolya (1996)
3      154      242       3  879138235  Kolya (1996)
4      306      242       5  876503793  Kolya (1996)


# **Step 3: Create User-Item Interaction Matrix**
We create a matrix where each row represents a user, each column represents a movie, and the values are the ratings.

In [3]:
# Create the user-item interaction matrix
user_item_matrix = data.pivot_table(index='user_id', columns='title', values='rating').fillna(0)

# Display the matrix
print(user_item_matrix.head())


title    'Til There Was You (1997)  1-900 (1994)  101 Dalmatians (1996)  \
user_id                                                                   
1                              0.0           0.0                    2.0   
2                              0.0           0.0                    0.0   
3                              0.0           0.0                    0.0   
4                              0.0           0.0                    0.0   
5                              0.0           0.0                    2.0   

title    12 Angry Men (1957)  187 (1997)  2 Days in the Valley (1996)  \
user_id                                                                 
1                        5.0         0.0                          0.0   
2                        0.0         0.0                          0.0   
3                        0.0         2.0                          0.0   
4                        0.0         0.0                          0.0   
5                        0.0        

# **Step 4: Convert the Matrix to a Sparse Matrix**
For efficiency, we convert the user-item interaction matrix to a sparse matrix.

In [4]:
from scipy.sparse import csr_matrix

# Convert the user-item matrix to a sparse matrix
user_item_sparse = csr_matrix(user_item_matrix.values)


# **Step 5: Calculate User Similarities**
We use cosine similarity to calculate the similarity between users.

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate the cosine similarity between users
user_similarity = cosine_similarity(user_item_sparse)


# **Step 6: Generate Movie Recommendations**
We define a function to generate movie recommendations for a given user based on the ratings of similar users.

In [6]:
import numpy as np

def get_recommendations(user_id, num_recommendations=5):
    # Get the user's ratings
    user_ratings = user_item_matrix.loc[user_id]

    # Get the similarity scores for the user
    user_similarities = user_similarity[user_id - 1]

    # Calculate the weighted sum of ratings from similar users
    weighted_ratings = np.dot(user_similarities, user_item_matrix.values) / np.array([np.abs(user_similarities).sum(axis=0)])

    # Create a DataFrame with the weighted ratings
    weighted_ratings_df = pd.DataFrame(weighted_ratings, index=user_item_matrix.columns, columns=['rating'])

    # Filter out movies the user has already rated
    unrated_movies = weighted_ratings_df[~user_ratings.astype(bool)]

    # Get the top N recommendations
    recommendations = unrated_movies.sort_values('rating', ascending=False).head(num_recommendations)

    return recommendations.index.tolist()


# **Step 7: Get Recommendations for a Specific User**
We use the get_recommendations function to get movie recommendations for a specific user.

In [7]:
# Get movie recommendations for a specific user
user_id = 1
recommendations = get_recommendations(user_id)
print(f"Recommendations for User {user_id}: {recommendations}")


Recommendations for User 1: ["Schindler's List (1993)", 'E.T. the Extra-Terrestrial (1982)', "One Flew Over the Cuckoo's Nest (1975)", 'English Patient, The (1996)', 'Scream (1996)']


In [8]:
# Get movie recommendations for a specific user
user_id = 7
recommendations = get_recommendations(user_id)
print(f"Recommendations for User {user_id}: {recommendations}")


Recommendations for User 7: ['Toy Story (1995)', 'Star Trek: First Contact (1996)', 'Rock, The (1996)', 'Aladdin (1992)', 'Titanic (1997)']


In [9]:
# Get movie recommendations for a specific user
user_id = 77
recommendations = get_recommendations(user_id)
print(f"Recommendations for User {user_id}: {recommendations}")


Recommendations for User 77: ['Back to the Future (1985)', 'Fugitive, The (1993)', 'Twelve Monkeys (1995)', "Schindler's List (1993)", 'Shawshank Redemption, The (1994)']
